In [ ]:
import pyarrow as pa
import pyarrow.ipc as ipc
import numpy as np
import ray

ray.init()

test_dir = "./tes"

paths = []

for i in range(10):
    data = [
        pa.array(np.arange(0, i+1, dtype=np.int64)),
        pa.array([i] * (i+1), type=pa.int64())
    ]
    batch = pa.RecordBatch.from_arrays(data, ['numbers', 'strings'])

    path = test_dir+"/"+str(i)
    paths.append(path)
    # Serialize RecordBatch to a file
    with pa.OSFile(path, 'wb') as sink:
        with ipc.new_stream(sink, batch.schema) as writer:
            writer.write_batch(batch)

In [ ]:
import ray

# 初始化 Ray
ray.init()

# 创建一个 Dataset
ds = ray.data.range(10)

from ray.data.datasource import FilenameProvider
class OriFilenameProvider(FilenameProvider):
    def __init__(self):
        pass
    def get_filename_for_block(self, block, task_index: int, block_index: int) -> str:
        return ""
from pyarrow import csv
# 将 Dataset 写入到 CSV 文件中
csv.WriteOptions(include_header=True)
ds.write_csv("output.csv",arrow_csv_args_fn=pyarrow..,concurrency=1,filename_provider=OriFilenameProvider(),try_create_dir=False)

In [ ]:
import posixpath

a = "/test/csv.out"
print(posixpath.join(a,""))

import pathlib
print(pathlib.Path(a).joinpath(""))

In [ ]:
import ray
from ray.data.datasource import Datasource, ReadTask
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.ipc as ipc
import numpy as np
from ray.data.block import BlockMetadata
from typing import Iterable
from ray.data.block import Block

class ReadArrowFilesDatasource(Datasource):
    def prepare_read(self, parallelism: int, files):
        def read_file(files) -> Iterable[Block]:
            for file in files:
                with pa.memory_map(file, 'rb') as source:
                    with ipc.open_stream(source) as reader:
                        tbl = pa.Table.from_batches([b for b in reader])
                        print(tbl.to_pandas())
                        yield tbl

        file_batches = np.array_split(files, np.ceil(len(files) / parallelism))

        meta = BlockMetadata(
            num_rows=None,
            size_bytes=None,
            schema=None,
            input_files=None,
            exec_stats=None,
        )
        read_tasks = [
            ReadTask(lambda p=file_batch: read_file(p), meta)
            for file_batch in file_batches
        ]
        return read_tasks



# 创建Dataset
custom_datasource = ReadArrowFilesDatasource()

dataset = ray.data.read_datasource(custom_datasource, files=paths)
dataset._lazy = False
# 当你调用 to_arrow_refs 时，每个ObjectRef将对应每个文件中的内容
arrow_refs = dataset.to_arrow_refs()


print(len(arrow_refs))

# for ref in arrow_refs:
#     print(ray.get(ref).to_pandas())
#     print("------------")


In [ ]:
print(dataset.schema())


print(dir(dataset.schema()),type(dataset.schema()))
for field in dataset.schema().types:
    print(field,type(field))
print(dataset.schema().base_schema,type(dataset.schema().base_schema))